In [ ]:
import json

from osm_bot_abstraction_layer.overpass_downloader import download_overpass_query
import osm_bot_abstraction_layer.osm_bot_abstraction_layer as osm_bot_abstraction_layer
from osm_bot_abstraction_layer.generic_bot_retagging import run_simple_retagging_task

# Checking OSM data befor the edit

In [ ]:
amentiy_hitchhiking_bench_download = """
/*

*/
[out:json][timeout:25];
// gather results
nwr["amenity"="hitchhiking_bench"];
// print results
out geom;
"""

file_path = './tmp/amentity_hitchhiking_bench.osm'
download_overpass_query(amentiy_hitchhiking_bench_download, file_path)

In [ ]:
with open(file_path, 'r') as file:
    data = json.load(file)

    print(len(data['elements']), "elements")

In [ ]:
for element in data['elements']:
    type = element['type']
    id = element['id']

    # Get the data for the current element
    print(f"Downloading data for {type} with id {id}")
    node_json = osm_bot_abstraction_layer.get_data(id, type)
    
    print(json.dumps(node_json, default=str, indent=4))

# Performing the edit

In [ ]:
def edit_element(tags):
    if "amenity" not in tags.keys():
        tags.pop('highway', None)
        tags["highway"] = "hitchhiking"
    
    if "amenity" in tags.keys() and tags["amenity"] == "bench":
        tags.pop('highway', None)
        tags.pop('amenity', None)
        tags["highway"] = "hitchhiking"
        tags["bench"] = "yes"
    
    return tags

highway_hitchhiking_bench_download = """
/*

*/
[out:xml][timeout:25];
// gather results
nwr["highway"="hitchhiking_bench"];
// print results
out geom;
"""

def main():
    run_simple_retagging_task(
        max_count_of_elements_in_one_changeset=500,
        objects_to_consider_query=highway_hitchhiking_bench_download,
        cache_folder_filepath='./cache',
        is_in_manual_mode=True,
        changeset_comment='Replacing undocumented and redundant hitchhiking=local tag with highway=hitchhiking. Part of the mechanical edit from https://wiki.openstreetmap.org/wiki/Mechanical_Edits/TillWenke/moving_hitchhiking_stops_to_%27highway%3Dhitchhiking%27_(mainly_in_Europe/Germany).',
        discussion_url='https://wiki.openstreetmap.org/wiki/Talk:Mechanical_Edits/TillWenke/moving_hitchhiking_stops_to_%27highway%3Dhitchhiking%27_(mainly_in_Europe/Germany)',
        osm_wiki_documentation_page='https://wiki.openstreetmap.org/wiki/Mechanical_Edits/TillWenke/moving_hitchhiking_stops_to_%27highway%3Dhitchhiking%27_(mainly_in_Europe/Germany)',
        edit_element_function=edit_element,
    )

main()